In [1]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Read the dataset, drop the columns which are unique for all users like IDs, distinguish the feature and target set

In [2]:
#importing the dataset
churn_df = pd.read_csv('Churn_Modelling.csv')
X = churn_df.iloc[:,3:13].values # Credit Score through Estimated Salary
y = churn_df.iloc[:, 13].values # Exited

In [3]:
# Encoding categorical (string based) data. Country: there are 3 options: France, Spain and Germany
# This will convert those strings into scalar values for analysis
print(X[:8,1], '... will now become: ')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


In [4]:
# We will do the same thing for gender. this will be binary in this dataset
print(X[:6,2], '... will now become: ')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


In [5]:
# Converting the string features into their own dimensions. Gender doesn't matter here because its binary
countryhotencoder = OneHotEncoder(categorical_features = [1]) # 1 is the country column
X = countryhotencoder.fit_transform(X).toarray()

C:\Users\mamirashraff\Downloads\WPy64-3720\python-3.7.2.amd64\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\mamirashraff\Downloads\WPy64-3720\python-3.7.2.amd64\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [7]:
X = X[:,1:] # Got rid of Spain as a dimension. It is still there through out inferences

### Divide the data set into Train and test sets

In [8]:
# Splitting the dataset into the Training and Testing set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

### Normalize the train and test data

In [9]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

### Initialize & build the model 

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [12]:
# Initializing the ANN
classifier = Sequential()

##### A hurestic tip is that the amount of nodes (dimensions) in your hidden layer should be the average of your input and output layers, which means that since we have 11 dimensions (representing Independent variables Note: Countries still compose only one dimension) and we are looking for a binary output, we calculate this to be  (11+1)÷2=6 .

##### The breakdown of the inputs for the first layer is as follows:
##### activiation: relu becasue we are in an input layer. uses the ReLu activation function for  ϕ 
##### input_dim: 11 because we span 11 dimensions in our input layer. This is needed for the first added layer. The subsequent layers's input dimensions can be inferred using the previously added layer's output dimension. The next hidden layer will know what to expect.

##### units: 6 nodes (number of nodes in hidden layer). Can think of this as number of nodes are in the next layer.

##### kernel_initializer: uniform the distribution with which we randomly initialize weights for the nodes in this layer.

In [13]:
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

Instructions for updating:
Colocations handled automatically by placer.


##### We are going to add another layer to this model because we want to implement Deep Learning, which is an artificial Neural network with many layers.
##### We will make our second hidden layer also have 6 nodes, just playing with the same arithmetic we used to determine the dimensions of the first hidden layer (average of your input and output layers)  (11+1)÷2=6 .

In [14]:
# Adding the second hidden layer
# Notice that we do not need to specify input dim. 
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform'))

##### Adding the output layer
##### The breakdown of the inputs for the output layer is as follows:
##### activiation: sigmoid becasue we are in an output layer. uses the Sigmoid activation function for  ϕ . This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.

##### input_dim: 11 because we span 11 dimensions in our input layer. This is needed for the first added layer. The subsequent layers's input dimensions can be inferred using the previously added layer's output dimension. The next hidden layer will know what to expect.

##### units: 1 node (number of nodes in hidden layer). Can think of this as number of nodes in the next layer.

##### kernel_initializer: uniform the distribution with which we randomly initialize weights for the nodes in this layer.

In [15]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform'))

##### If we want more than two categories, then we will need to change
##### 1) the units parameter to match the desired category count

##### 2) the activation field to softmax. Basically a sigmoid function but applied to a dependent variable that has more than 2 categories.

### Optimize the model

##### Compiling the Neural Network
##### Basically applying Stochastic Gradient descent on the whole Neural Network. We are Tuning the individual weights on each neuron.

##### The breakdown of the inputs for compiling is as follows:
##### optimizer: adam The algorithm we want to use to find the optimal set of weights in the neural networks. Adam is a very efficeint variation of Stochastic Gradient Descent.

##### loss: binary_crossentropy This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is Binary, it is binary_crossentropy. If Categorical, then it is called categorical_crossentropy

##### metrics: [accuracy] The accuracy metrics which will be evaluated(minimized) by the model. Used as accuracy criteria to imporve model performance.

##### kernel_initializer: uniform the distribution with which we randomly initialize weights for the nodes in this layer.

In [16]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

##### Fitting the Neural Network
##### This is where we will be fitting the ANN to our training set.

##### The breakdown of the inputs for compiling is as follows:
##### X_train The independent variable portion of the data which needs to be fitted with the model.

##### Y_train The output portion of the data which the model needs to produce after fitting.

##### batch_size: How often we want to back-propogate the error values so that individual node weights can be adjusted.

##### nb_epochs: The number of times we want to run the entire test data over again to tune the weights. This is like the fuel of the algorithm.

In [19]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 1s 92us/step - loss: 0.3917 - acc: 0.8366
Epoch 2/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.3919 - acc: 0.8374
Epoch 3/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.3920 - acc: 0.8376
Epoch 4/100
8000/8000 [==============================] - 1s 91us/step - loss: 0.3917 - acc: 0.8371
Epoch 5/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.3920 - acc: 0.8365
Epoch 6/100
8000/8000 [==============================] - 1s 92us/step - loss: 0.3917 - acc: 0.8366
Epoch 7/100
8000/8000 [==============================] - 1s 136us/step - loss: 0.3918 - acc: 0.8367
Epoch 8/100
8000/8000 [==============================] - 1s 154us/step - loss: 0.3922 - acc: 0.8381
Epoch 9/100
8000/8000 [==============================] - 1s 150us/step - loss: 0.3917 - acc: 0.8382
Epoch 10/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.3921 - acc: 0.8367
Epoch

##### Testing the ANN
##### Predicting the Test set results
##### This shows the probability of a customer leaving given the testing data. Each row in X_test corresponds to a row in Y_test

In [20]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.11874357]
 [0.30970865]
 [0.1136061 ]
 ...
 [0.15000975]
 [0.11706641]
 [0.17748287]]


### Predict the results using 0.5 as a threshold

##### To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.

In [21]:
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


### Print the Accuracy score and confusion matrix

##### Making the Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1546   49]
 [ 261  144]]


In [23]:
classifier.evaluate(X_test,y_test)

2000/2000 [==============================] - 0s 53us/step


[0.39483959329128265, 0.845]

##### 84.5% of testing data was classified correctly